In [51]:
import pandas as pd

In [52]:
df=pd.read_csv('train.csv')

In [53]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
df['text'] = df['text'] + " " + df['title']
del df['title']
del df['author']
del df['id']

In [55]:
df=df.dropna()

In [56]:
df.reset_index(drop=True, inplace=True)

In [57]:
import tensorflow as tf

In [58]:
tf.__version__

'1.14.0'

In [59]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [60]:
voc_size=10000

In [61]:
import nltk
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re,string,unicodedata

In [62]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tanvi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
### Dataset Preprocessing
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
    
# Removing URL's
def remove_url(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_url(text)
    text = remove_stopwords(text)
    return text

#Apply function on review column
df['text']=df['text'].apply(denoise_text)

In [64]:
df

,text,label
0,House Dem Aide: Didn’t Even See Comey’s Letter...,1
1,Ever get feeling life circles roundabout rathe...,0
2,"Truth Might Get Fired October 29, 2016 tension...",1
3,Videos 15 Civilians Killed Single US Airstrike...,1
4,Print Iranian woman sentenced six years prison...,1
...,...,...
20198,Rapper T. I. unloaded black celebrities met Do...,0
20199,Green Bay Packers lost Washington Redskins Wee...,0
20200,Macy’s today grew union several great names Am...,0
20201,"NATO, Russia Hold Parallel Exercises Balkans 1...",1


In [65]:
corpus = df['text'].tolist()

In [66]:
corpus[0]

'House Dem Aide: Didn’t Even See Comey’s Letter Jason Chaffetz Tweeted Darrell Lucus October 30, 2016 Subscribe Jason Chaffetz stump American Fork, Utah image courtesy Michael Jolley, available Creative Commons-BY license) apologies Keith Olbermann, doubt Worst Person World week–FBI Director James Comey. according House Democratic aide, looks like also know second-worst person well. turns Comey sent now-infamous letter announcing FBI looking emails may related Hillary Clinton’s email server, ranking Democrats relevant committees didn’t hear Comey. found via tweet one Republican committee chairmen. know, Comey notified Republican chairmen Democratic ranking members House Intelligence, Judiciary, Oversight committees agency reviewing emails recently discovered order see contained classified information. long letter went out, Oversight Committee Chairman Jason Chaffetz set political world ablaze tweet. FBI Dir informed me, "The FBI learned existence emails appear pertinent investigation."

In [67]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 


In [68]:
onehot_repr[0]

[6170,
 7378,
 638,
 6832,
 5505,
 4652,
 8964,
 61,
 1004,
 566,
 96,
 6956,
 2105,
 8321,
 3783,
 3151,
 730,
 1004,
 566,
 3277,
 1508,
 7822,
 9162,
 1750,
 6042,
 4569,
 3328,
 1453,
 8263,
 3648,
 5186,
 6009,
 247,
 7616,
 8092,
 6238,
 8667,
 3861,
 3202,
 4789,
 609,
 318,
 960,
 7497,
 6170,
 5513,
 638,
 9302,
 5917,
 1315,
 7316,
 9545,
 8667,
 3861,
 8305,
 3654,
 960,
 1862,
 5032,
 6282,
 61,
 6153,
 8921,
 6512,
 8018,
 7147,
 9627,
 2489,
 6907,
 5322,
 2305,
 414,
 3622,
 4174,
 1644,
 6832,
 8878,
 960,
 9631,
 6893,
 7284,
 9417,
 1527,
 3293,
 910,
 7316,
 960,
 704,
 1527,
 910,
 5513,
 414,
 6059,
 6170,
 4214,
 8292,
 7328,
 1644,
 7623,
 6586,
 8018,
 4215,
 2989,
 9392,
 4652,
 2819,
 5177,
 6036,
 4285,
 61,
 3940,
 941,
 7328,
 3293,
 5113,
 1004,
 566,
 8434,
 8365,
 3202,
 3073,
 7284,
 8921,
 2566,
 5783,
 9592,
 4582,
 8921,
 1746,
 1818,
 8018,
 1482,
 2510,
 5390,
 8175,
 1050,
 3111,
 1004,
 566,
 7266,
 8321,
 1968,
 3151,
 5470,
 7316,
 8175,
 960,


In [69]:
sent_length=300
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[ 937 1715  243 ... 1004  566   96]
 [2167 6607 4538 ... 6054  144 4382]
 [4214 7623 8832 ... 9859 9544 9079]
 ...
 [1199 3644  574 ... 1396 1432  559]
 [   0    0    0 ... 4142 7710 1856]
 [5847 7777 8988 ... 9035 7169 7037]]


In [70]:
len(embedded_docs)

20203

In [71]:
embedded_docs[20201]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 7794, 6250, 8133, 4142, 7710, 1856, 3721, 2702,
       3151, 8588, 5532, 9531, 4699, 7794, 3350, 5799, 4142, 4699, 7710,
        574, 7222, 9695, 2940, 1107,   42, 2087, 4910, 3450, 7527, 7794,
       6165, 4699, 3058, 1320, 8002, 2206, 1199, 11

In [72]:
## Creating model
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None


In [76]:
import numpy as np
y_final=df['label']
X_final=np.array(embedded_docs)

In [77]:
X_final.shape,y_final.shape

((20203, 300), (20203,))

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [79]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 13536 samples, validate on 6667 samples
Epoch 1/10
13536/13536 [==============================] - 760s 56ms/sample - loss: 0.2027 - acc: 0.9205 - val_loss: 0.0765 - val_acc: 0.9738
Epoch 2/10
13536/13536 [==============================] - 328s 24ms/sample - loss: 0.0273 - acc: 0.9919 - val_loss: 0.0830 - val_acc: 0.9721
Epoch 3/10
13536/13536 [==============================] - 302s 22ms/sample - loss: 0.0066 - acc: 0.9979 - val_loss: 0.0948 - val_acc: 0.9688
Epoch 4/10
13536/13536 [==============================] - 476s 35ms/sample - loss: 0.0020 - acc: 0.9997 - val_loss: 0.1276 - val_acc: 0.9738
Epoch 5/10
13536/13536 [==============================] - 342s 25ms/sample - loss: 4.6820e-04 - acc: 0.9999 - val_loss: 0.1393 - val_acc: 0.9724
Epoch 6/10
13536/13536 [==============================] - 378s 28ms/sample - loss: 1.1476e-04 - acc: 1.0000 - val_loss: 0.1474 - val_acc: 0.9721
Epoch 7/10
13536/13536 [==============================] - 358s 26ms/sample - loss: 7.1301e-05 - a

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)